1.
2. book(book_id, title, publisher_id, genre, publication_year)
FK: publisher_id → publisher.publisher_id
2. publisher(publisher_id, name, location, founding_year)
3. author(author_id, name, nationality, birth_year)
4. writes(author_id, book_id)
FK: author_id → author.author_id
FK: book_id → book.book_id
5. library_branch(branch_id, name, location, established_year)
6. copy(copy_id, book_id, branch_id, status)
FK: book_id → book.book_id
FK: branch_id → library_branch.branch_id
7. borrower(borrower_id, name, email, join_date)
8. loan(loan_id, copy_id, borrower_id, loan_date, due_date, return_date)
FK: copy_id → copy.copy_id
FK: borrower_id → borrower.borrower_id
9. genre(genre_id, genre_name)
10. book_genre(book_id, genre_id)
FK: book_id → book.book_id
FK: genre_id → genre.genre_id


```sql
CREATE TABLE book (
    book_id INT,
    title VARCHAR,
    publisher_id INT,
    genre VARCHAR,
    publication_year INT,
    PRIMARY KEY (book_id),
);

CREATE TABLE publisher (
    publisher_id INT,
    name VARCHAR,
    location VARCHAR,
    founding_year INT,
    PRIMARY KEY (publisher_id)
);

CREATE TABLE author (
    author_id INT,
    name VARCHAR,
    nationality VARCHAR,
    birth_year INT,
    PRIMARY KEY (author_id)
);

CREATE TABLE writes (
    author_id INT,
    book_id INT,
    PRIMARY KEY (author_id, book_id),
);

CREATE TABLE library_branch (
    branch_id INT,
    name VARCHAR,
    location VARCHAR,
    established_year INT,
    PRIMARY KEY (branch_id)
);

CREATE TABLE copy (
    copy_id INT,
    book_id INT,
    branch_id INT,
    status VARCHAR,
    PRIMARY KEY (copy_id),
);

CREATE TABLE borrower (
    borrower_id INT,
    name VARCHAR,
    email VARCHAR,
    join_date DATE,
    PRIMARY KEY (borrower_id)
);

CREATE TABLE loan (
    loan_id INT,
    copy_id INT,
    borrower_id INT,
    loan_date DATE,
    due_date DATE,
    return_date DATE,
    PRIMARY KEY (loan_id),
);

CREATE TABLE genre (
    genre_id INT,
    genre_name VARCHAR,
    PRIMARY KEY (genre_id)
);

CREATE TABLE book_genre (
    book_id INT,
    genre_id INT,
    PRIMARY KEY (book_id, genre_id),
);
```


Find the ID and name of each author who has written at least one book in the "Science" genre.
```sql
SELECT AUTHOR.author_id, AUTHOR.name
FROM author AUTHOR
JOIN writes WRITES
ON AUTHOR.author_id = WRITES.author_id
JOIN book_genre BG
ON WRITES.book_id = BG.book_id
JOIN genre G
ON BG.genre_id = G.genre_id
WHERE G.genre_name = 'Science';
```
Find the ID and name of each borrower who has borrowed at least one book from the "Downtown" library branch.
```sql
SELECT BORROWER.borrower_id, BORROWER.name
FROM borrower BORROWER
JOIN loan LOAN
ON BORROWER.borrower_id = LOAN.borrower_id
JOIN copy COPY
ON LOAN.copy_id = COPY.copy_id
JOIN library_branch LB
ON COPY.branch_id = LB.branch_id
WHERE LB.name = 'Downtown';
```
Find the ID and name of each borrower who has borrowed at least one book published by "Penguin Books".
```sql
SELECT BORROWER.borrower_id, BORROWER.name
FROM borrower BORROWER
JOIN loan LOAN
    ON BORROWER.borrower_id = LOAN.borrower_id
JOIN copy COPY
    ON LOAN.copy_id = COPY.copy_id
JOIN book BOOK
    ON COPY.book_id = BOOK.book_id
JOIN publisher PUB
    ON BOOK.publisher_id = PUB.publisher_id
WHERE PUB.name = 'Penguin Books';
```
Find the ID and name of each borrower who has not returned any books by the due date. Note: the query should not return borrowers who unless every book they borrowed is overdue.
```sql
SELECT BORROWER.borrower_id, BORROWER.name
FROM borrower BORROWER
JOIN loan LOAN
    ON BORROWER.borrower_id = LOAN.borrower_id
GROUP BY BORROWER.borrower_id, BORROWER.name
HAVING COUNT(CASE WHEN LOAN.return_date IS NULL OR LOAN.return_date > LOAN.due_date THEN 1 END) = COUNT(*);
```
Find the ID and name of each book that has never been borrowed.
```sql
SELECT BOOK.book_id, BOOK.title
FROM book BOOK
LEFT JOIN copy COPY
    ON BOOK.book_id = COPY.book_id
LEFT JOIN loan LOAN
    ON COPY.copy_id = LOAN.copy_id
WHERE LOAN.loan_id IS NULL;
```
Find the ID and name of each borrower who has borrowed at least one book from every genre.
```sql
SELECT BORROWER.borrower_id, BORROWER.name
FROM borrower BORROWER
JOIN loan LOAN
    ON BORROWER.borrower_id = LOAN.borrower_id
JOIN copy COPY
    ON LOAN.copy_id = COPY.copy_id
JOIN book_genre BG
    ON COPY.book_id = BG.book_id
JOIN genre G
    ON BG.genre_id = G.genre_id
GROUP BY BORROWER.borrower_id, BORROWER.name
HAVING COUNT(DISTINCT G.genre_id) = (SELECT COUNT(*) FROM genre);
```
Find the name of each borrower who has borrowed all "Fiction" books.
```sql
SELECT BORROWER.name
FROM borrower BORROWER
JOIN loan LOAN ON BORROWER.borrower_id = LOAN.borrower_id
JOIN copy COPY ON LOAN.copy_id = COPY.copy_id
JOIN book BOOK ON COPY.book_id = BOOK.book_id
JOIN book_genre BG ON BOOK.book_id = BG.book_id
JOIN genre G ON BG.genre_id = G.genre_id
WHERE G.genre_name = 'Fiction'
GROUP BY BORROWER.name
HAVING COUNT(DISTINCT BOOK.book_id) = (
    SELECT COUNT(DISTINCT BOOK.book_id)
    FROM book BOOK
    JOIN book_genre BG ON BOOK.book_id = BG.book_id
    JOIN genre G ON BG.genre_id = G.genre_id
    WHERE G.genre_name = 'Fiction'
);
```
List the ID and name of each publisher whose total number of published books exceeds the average number of books published by all publishers.
```sql
SELECT PUB.publisher_id, PUB.name
FROM publisher PUB
JOIN book BOOK
    ON PUB.publisher_id = BOOK.publisher_id
GROUP BY PUB.publisher_id, PUB.name
HAVING COUNT(BOOK.book_id) > (SELECT AVG(book_count) FROM (SELECT COUNT(*) AS book_count FROM book GROUP BY publisher_id) AS avg_books);
```





